# CartPole
---
>continous 한 action 환경 중에서 가장 단순한 환경이다. 매 프레임마다 reward 를 제공하는 dense reward 환경이라서 때문에 학습이 매우 용이하다. 
또한 terminal condition 은 카트가 일정 범위 넘어가거나 막대가 일정 각도 이상되는 것이기 때문에 명확하다. DQN 으로 할 때 step 마다 학습하지 않고 데이터만 쌓는다. 나중에 학습과 타겟 신경망 업데이트 하는것이 최적



* state : 카트 위치, 카트 속력, 막대 각도, 막대 각속도 - continuous
* action : 카트를 오른쪽/왼쪽으로 push
* reward : 매 프레임마다 +1, episode 가 끝날때 임의로 보상제공 가능
---

In [1]:
import tensorflow as tf
import numpy as np
import gym
import random
from collections import deque
import dqn

In [2]:
'''
환경 생성
'''
env = gym.make('CartPole-v1')

In [3]:
size_s = env.observation_space.shape[0]
size_out = env.action_space.n
gamma = .95
print(size_out)

2


In [4]:
'''
학습 네트워크 -> target 네트워크 로 복사하는 함수
''' 
def copy(*, dest_scope_name = 'target', src_scope_name = 'main') :
    op_holder = []
    
    src_vars = tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES, scope = src_scope_name)
    dest_vars = tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES, scope = dest_scope_name)
    
    for src_var, dest_var in zip(src_vars, dest_vars) :
        op_holder.append(dest_var.assign(src_var))
        
    return op_holder


In [5]:
'''
replay experience, 공급된 batch 로 학습
'''
def replay_train(mainDQN, targetDQN, train_batch) :
    x_stack = np.empty(0).reshape(0, size_s)
    y_stack = np.empty(0).reshape(0, size_out)
    
    for state, action, reward, next_state, done in train_batch :
        
        # 현재 Q 는 main 네트워크에서 도출
        Q = mainDQN.predict(state)
        
        # terminal step 이면 음의 보상
        if done :
            Q[0, action] = -100
        
        # target Q 는 target 네트워크에서 도출 - 자기상관도 감소
        else :
            # 그냥 DQN 업데이트 - 타겟 Q 값은 해당 state 에서 max 값을 취한다.
            #Q[0, action] = reward + gamma * np.max(targetDQN.predict(next_state))
            
            # Double DQN 업데이트 - 타겟 Q 값은 main network 로부터 도출한다.
            a_target = np.argmax(mainDQN.predict(next_state))
            Q[0, action] = reward + gamma * targetDQN.predict(next_state)[0][a_target]
            
        
        # x 는 현재 Q
        # y 는 target
        x_stack = np.vstack([x_stack, state])
        y_stack = np.vstack([y_stack, Q])
        
    return mainDQN.update(x_stack, y_stack)

In [6]:
total_episode = 1000
replay_buffer = deque()
ep = 0
step = 0
#saver = tf.train.Saver()
#save_file = 'C:\\Users\\김민수\\Documents\\GitHub\\RL\\vars\\cartpole_dqn'
mean_reward = deque()

with tf.Session() as sess:
      
    '''
    - 네트워크 2 개 생성
    - 하나는 실제 학습을 하는 학습 네트워크
    - 다른 하나는 일시적으로 고정되어 학습의 목표가 되는 target 네트워크
    '''
    mainDQN = dqn.DQN(sess, size_s, size_out, 8, 6, name='main')
    targetDQN = dqn.DQN(sess, size_s, size_out, 8, 6, name='target')
    
    tf.global_variables_initializer().run()
    
    # 처음 시작 시 네트워크 복사
    copy_ops = copy(dest_scope_name='targetDQN', src_scope_name='mainDQN')
    sess.run(copy_ops)
    
    state = env.reset()
    
    for episode in range(total_episode) :
        e = 1. / ((episode / 20) + 1)    # 점점 감소하는 explore 
        done = False
        step_count = 0
        state = env.reset()
        
        while not done :
             
            # 액션 선택
            if np.random.rand(1) < e :
                action = env.action_space.sample()
            else :
                action = np.argmax(mainDQN.predict(state))
                    
            next_state, reward, done, _ = env.step(action)
                
            if done : 
                reward = -100
                
                
            # 학습하지않고 임시메모리에 저장한다.
            replay_buffer.append((state, action, reward, next_state, done))
            if(len(replay_buffer) > 50000) :
                replay_buffer.popleft()
            
            state = next_state
            step_count += 1
            
            if(step_count > 1000) :
                break
        
        
        print ('episode : {}, steps : {}'.format(episode, step_count))
                
        '''
        파라미터 로컬에 저장.
        '''
        if(len(mean_reward) < 10):
            mean_reward.append(step_count)
        else:
            mean_reward.popleft()
            
            '''
            if(np.mean(mean_reward) > 800):
                saver.save(sess, save_file)
                break
            '''
                
        # 10 회의 에피소드가 끝나면 임시메모리에서 과거 데이터를 랜덤으로 뽑아 학습한다.
        if(episode % 10 == 1) :
            for _ in range(50) :
                minibacth = random.sample(replay_buffer, 10)
                loss, _ = replay_train(mainDQN, targetDQN, minibacth)
                        
            print('loss :', loss)
            sess.run(copy_ops)   # 목표가 되는 네트워크를 업데이트한다.
                    
    env.close()

episode : 0, steps : 10
episode : 1, steps : 24
loss : 489.00653
episode : 2, steps : 34
episode : 3, steps : 28
episode : 4, steps : 15
episode : 5, steps : 13
episode : 6, steps : 11
episode : 7, steps : 10
episode : 8, steps : 15
episode : 9, steps : 14
episode : 10, steps : 14
episode : 11, steps : 15
loss : 2.6621509
episode : 12, steps : 39
episode : 13, steps : 39
episode : 14, steps : 51
episode : 15, steps : 56
episode : 16, steps : 60
episode : 17, steps : 35
episode : 18, steps : 152
episode : 19, steps : 20
episode : 20, steps : 39
episode : 21, steps : 31
loss : 1.9861908
episode : 22, steps : 12
episode : 23, steps : 9
episode : 24, steps : 8
episode : 25, steps : 12
episode : 26, steps : 9
episode : 27, steps : 13
episode : 28, steps : 9
episode : 29, steps : 10
episode : 30, steps : 13
episode : 31, steps : 12
loss : 4.4117866
episode : 32, steps : 42
episode : 33, steps : 82
episode : 34, steps : 42
episode : 35, steps : 61
episode : 36, steps : 60
episode : 37, steps 

KeyboardInterrupt: 